# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/emelyzelaya/Desktop/Challenge 6/Starter_Code 14/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,16.43,89,99,4.38,AU,1704747284
1,1,kingston,17.9970,-76.7936,30.53,70,20,9.26,JM,1704747066
2,2,olbia,40.9214,9.4856,11.03,87,75,6.17,IT,1704747284
3,3,alzamay,55.5562,98.6644,-7.94,89,100,1.53,RU,1704747284
4,4,kidal,18.4411,1.4078,23.74,12,0,4.60,ML,1704747285


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_city = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")

# Display the map
map_plot_city

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27))
wind_condition = (city_data_df["Wind Speed"] < 4.5)
cloud_condition = (city_data_df["Cloudiness"] == 0)

city_data_fil = city_data_df.loc[(temp_condition & wind_condition & cloud_condition), :]


# Drop any rows with null values
city_final = city_data_fil.dropna()

# Display sample data
city_final

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,georgetown,5.4112,100.3354,25.96,97,20,1.03,MY,1704747097
77,77,suva,-18.1416,178.4415,26.71,94,75,1.54,FJ,1704747304
81,81,antofagasta,-23.6500,-70.4000,22.60,69,24,3.18,CL,1704747306
91,91,howard springs,-12.4970,131.0470,26.95,95,100,2.01,AU,1704747309
97,97,cabo san lucas,22.8909,-109.9124,24.31,47,40,2.06,MX,1704746865
...,...,...,...,...,...,...,...,...,...,...
540,540,puerto vallarta,20.6204,-105.2307,24.95,78,99,0.89,MX,1704747316
541,541,forster,-32.1667,152.5167,23.54,93,100,2.02,AU,1704747441
542,542,taoudenni,22.6783,-3.9836,22.53,23,0,3.51,ML,1704747441
551,551,bontang,0.1333,117.5000,24.25,91,100,1.49,ID,1704747443


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
30,georgetown,MY,5.4112,100.3354,97,
77,suva,FJ,-18.1416,178.4415,94,
81,antofagasta,CL,-23.6500,-70.4000,69,
91,howard springs,AU,-12.4970,131.0470,95,
97,cabo san lucas,MX,22.8909,-109.9124,47,
...,...,...,...,...,...,...
540,puerto vallarta,MX,20.6204,-105.2307,78,
541,forster,AU,-32.1667,152.5167,93,
542,taoudenni,ML,22.6783,-3.9836,23,
551,bontang,ID,0.1333,117.5000,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accomodation.hotel"]
apiKey = geoapify_key

params = {
    "categories": categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
   longitude = hotel_df.loc[index, "Lng"]
   latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
   params["filter"] = f'circle:{longitude},{latitude},{radius}'
   params["bias"] = f'proximity:{longitude},{latitude}'
   params["categories"] = f'accommodation.hotel'
    
    # Set base URL
   base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
   name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
   name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
   except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
   print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
georgetown - nearest hotel: Page 63 hostel
suva - nearest hotel: Extension 68
antofagasta - nearest hotel: Hotel Colón
howard springs - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
alice springs - nearest hotel: Aurora Alice Springs
antsiranana - nearest hotel: Central
akjoujt - nearest hotel: فندق فاطمة
kalmunai - nearest hotel: Saji new food
mananjary - nearest hotel: Ideal Hotel Annexe
raja - nearest hotel: No hotel found
chui - nearest hotel: Alerces
neelankarai - nearest hotel: Hotel Park Plaza Chennai OMR
kuta - nearest hotel: Ida Hotel Bali
kimbe - nearest hotel: Kimbe Bay Hotel
pangobilian - nearest hotel: No hotel found
riosucio - nearest hotel: Hotel Real Plaza
la reforma - nearest hotel: No hotel found
toamasina - nearest hotel: Androranga
kawm umbu - nearest hotel: No hotel found
malvan - nearest hotel: Hotel Silver Sand
mersing - nearest hotel: Mersing Inn Hotel
antsohihy - nearest hotel: Escale
sangmelima - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
30,georgetown,MY,5.4112,100.3354,97,Page 63 hostel
77,suva,FJ,-18.1416,178.4415,94,Extension 68
81,antofagasta,CL,-23.6500,-70.4000,69,Hotel Colón
91,howard springs,AU,-12.4970,131.0470,95,No hotel found
97,cabo san lucas,MX,22.8909,-109.9124,47,Comfort Rooms
...,...,...,...,...,...,...
540,puerto vallarta,MX,20.6204,-105.2307,78,Buenaventura Grand
541,forster,AU,-32.1667,152.5167,93,No hotel found
542,taoudenni,ML,22.6783,-3.9836,23,No hotel found
551,bontang,ID,0.1333,117.5000,91,Hotel Sederhana


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)